# 文本相似度匹配

### 背景介绍
文本语义匹配是自然语言处理中一个重要的基础问题，NLP 领域的很多任务都可以抽象为文本匹配任务。例如，信息检索可以归结为查询项和文档的匹配，问答系统可以归结为问题和候选答案的匹配，对话系统可以归结为对话和回复的匹配。语义匹配在搜索优化、推荐系统、快速检索排序、智能客服上都有广泛的应用。如何提升文本匹配的准确度，是自然语言处理领域的一个重要挑战。

- 信息检索：在信息检索领域的很多应用中，都需要根据原文本来检索与其相似的其他文本，使用场景非常普遍。
- 新闻推荐：通过用户刚刚浏览过的新闻标题，自动检索出其他的相似新闻，个性化地为用户做推荐，从而增强用户粘性，提升产品体验。
- 智能客服：用户输入一个问题后，自动为用户检索出相似的问题和答案，节约人工客服的成本，提高效率。

让我们来看一个简单的例子，比较各候选句子哪句和原句语义更相近：

- 原句：“车头如何放置车牌”
- 比较句1：“前牌照怎么装”
- 比较句2：“如何办理北京车牌”
- 比较句3：“后牌照怎么装”

比较结果：

- 比较句1与原句，虽然句式和语序等存在较大差异，但是所表述的含义几乎相同
- 比较句2与原句，虽然存在“如何” 、“车牌”等共现词，但是所表述的含义完全不同
- 比较句3与原句，二者讨论的都是如何放置车牌的问题，只不过一个是前牌照，另一个是后牌照。二者间存在一定的语义相关性
- 所以语义相关性，句1大于句3，句3大于句2，这就是语义匹配。

### 数据说明
LCQMC数据集比释义语料库更通用，因为它侧重于意图匹配而不是释义。LCQMC数据集包含 260,068 个带有人工标注的问题对。

- 包含 238,766 个问题对的训练集
- 包含 8,802 个问题对的开发集
- 包含 12,500 个问题对的测试集

### 评估方式
使用准确率Accuracy来评估，即：

    准确率(Accuracy)=预测正确的条目数/预测总条目数

也可以使用文本相似度与标签的皮尔逊系数进行评估，不匹配的文本相似度应该更低。

## 数据集读取

In [1]:
import pandas as pd

def load_lcqmc():
    '''LCQMC文本匹配数据集
    '''
    train = pd.read_csv('https://mirror.coggle.club/dataset/LCQMC.train.data.zip', 
            sep='\t', names=['query1', 'query2', 'label'])

    valid = pd.read_csv('https://mirror.coggle.club/dataset/LCQMC.valid.data.zip', 
            sep='\t', names=['query1', 'query2', 'label'])

    test = pd.read_csv('https://mirror.coggle.club/dataset/LCQMC.test.data.zip', 
            sep='\t', names=['query1', 'query2', 'label'])

    return train, valid, test

## 文本数据分析
步骤1：分析赛题文本长度，相似文本对与不相似文本对的文本长度是否存在差异？

步骤2：分析赛题单词和字符个数，在所有文本中包含多少个单词（用jieba进行分析）和字符？

In [2]:
train, valid, test = load_lcqmc()

In [3]:
len(train)

238766

In [4]:
train.head(10)

,query1,query2,label
0,喜欢打篮球的男生喜欢什么样的女生,爱打篮球的男生喜欢什么样的女生,1
1,我手机丢了，我想换个手机,我想买个新手机，求推荐,1
2,大家觉得她好看吗,大家觉得跑男好看吗？,0
3,求秋色之空漫画全集,求秋色之空全集漫画,1
4,晚上睡觉带着耳机听音乐有什么害处吗？,孕妇可以戴耳机听音乐吗?,0
5,学日语软件手机上的,手机学日语的软件,1
6,打印机和电脑怎样连接，该如何设置,如何把带无线的电脑连接到打印机上,0
7,侠盗飞车罪恶都市怎样改车,侠盗飞车罪恶都市怎么改车,1
8,什么花一年四季都开,什么花一年四季都是开的,1
9,看图猜一电影名,看图猜电影！,1


In [5]:
train['query1'].apply(lambda x: len(x)).max()

49

In [6]:
train['query2'].apply(lambda x: len(x)).max()

131

# Bert模型

In [7]:
from transformers import (
    BertTokenizer, 
    BertForSequenceClassification,
    AdamW,
    get_linear_schedule_with_warmup
)

import torch
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import accuracy_score

In [8]:
model_path= r"D:\code\models\bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(model_path)

In [9]:
# 定义数据类
class MyDataset(Dataset):
    """构建数据集
    """
    def __init__(self, encodings, labels):
        """初始化函数
        """
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        """获取下标idx的样本
        """
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(int(self.labels[idx]))
        return item
    
    def __len__(self):
        # NOTE:这里必须是labels，不能是encodings，因为len(encodings)=3
        return len(self.labels)
        
        

In [10]:
train_encoding = tokenizer(train['query1'].tolist(), train['query2'].tolist(), truncation=True, padding=True, max_length=150)
valid_encoding = tokenizer(valid['query1'].tolist(), valid['query2'].tolist(), truncation=True, padding=True, max_length=150)
test_encoding = tokenizer(test['query1'].tolist(), test['query2'].tolist(), truncation=True, padding=True, max_length=150)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


In [11]:
len(train_encoding)

3

In [12]:
len(train_encoding['input_ids'][0])

150

In [13]:
# ans = tokenizer(['你好'], ['河北'])
# ans

In [14]:
tokenizer.decode(train_encoding['input_ids'][0], skip_special_tokens=True)

'喜 欢 打 篮 球 的 男 生 喜 欢 什 么 样 的 女 生 爱 打 篮 球 的 男 生 喜 欢 什 么 样 的 女 生'

In [15]:
train_dataset = MyDataset(train_encoding, train['label'].tolist())
valid_daatset = MyDataset(valid_encoding, valid['label'].tolist())
test_dataset  = MyDataset(test_encoding, test['label'].tolist())

In [16]:
train_dataset[1]

{'input_ids': tensor([ 101, 2769, 2797, 3322,  696,  749, 8024, 2769, 2682, 2940,  702, 2797,
         3322,  102, 2769, 2682,  743,  702, 3173, 2797, 3322, 8024, 3724, 2972,
         5773,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0, 

In [17]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=1, shuffle=True)
valid_dataloader = DataLoader(dataset=valid_daatset, batch_size=1, shuffle=True)
test_dataloader  = DataLoader(dataset=test_dataset, batch_size=1, shuffle=True) 

In [18]:
train_dataloader

In [19]:
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at D:\code\models\bert-base-chinese and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# 超参数
epochs = 4
lr = 1e-5
total_steps = 1 * len(train_dataloader)
optim = AdamW(model.parameters(), lr=lr)
schedule = get_linear_schedule_with_warmup(optimizer=optim, num_warmup_steps=0, num_training_steps=total_steps)

D:\Python\envs\py310\lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [33]:
def train(epoch):
    model.train()
    
    iter_num = 0
    total_loss = 0
    total_iter = len(train_dataloader)
    for batch in train_dataloader:
        optim.zero_grad()  # 梯度清零
        
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        token_type_ids = batch['token_type_ids']
        label = batch['labels']

        output = model(input_ids, 
                       attention_mask=attention_mask, 
                       labels=label)
        loss = output[0]
        total_loss += loss.item()
        
        loss.backward()   # 后向传播
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # 梯度裁剪
        optim.step()      # 参数更新
        schedule.step()
        
        iter_num += 1
        if iter_num % 100 == 0:
            print("epoth: %d, iter_num: %d, loss: %.4f, %.2f%%" % (epoch, iter_num, loss.item(), iter_num/total_iter*100))
        
    print("Epoch: %d, Average training loss: %.4f"%(epoch, total_loss/len(train_loader)))

In [34]:
for i in range(epochs):
    print('------------ epochs:',i)
    train(i)

------------epochs: 0
epoth: 0, iter_num: 100, loss: 2.5876, 0.04%
epoth: 0, iter_num: 200, loss: 0.2299, 0.08%
epoth: 0, iter_num: 300, loss: 0.0272, 0.13%
epoth: 0, iter_num: 400, loss: 0.0014, 0.17%
epoth: 0, iter_num: 500, loss: 0.5963, 0.21%
epoth: 0, iter_num: 600, loss: 1.0480, 0.25%
epoth: 0, iter_num: 700, loss: 0.0183, 0.29%


KeyboardInterrupt: 

# SimCSE模型